In [ ]:
!pip install transformers datasets torch

In [3]:
import prompts

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [5]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", token="TOKEN", torch_dtype=torch.bfloat16, load_in_4bit=True )
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", token="TOKEN")

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
)

In [7]:
def generate(prompt):
  messages = [ {"role": "user", "content": prompt} ]
  outputs = pipe(
    messages,
    max_new_tokens=800,
    pad_token_id=pipe.tokenizer.eos_token_id
  )
  response = outputs[0]["generated_text"][-1]
  return response["content"]

# CoVe steps

In [8]:
def generate_baseline_response(original_sentence):

  prompt = prompts.BASELINE_PROMPT.format(
      input=original_sentence
  )

  baseline_response = generate(prompt)
  response_json = json.loads(baseline_response)["rephrased_sentence"]

  return response_json

In [9]:
def generate_verification_questions(original_sentence, baseline_response):

  prompt = prompts.VERIFICATION_QUESTIONS_PROMPT.format(
    original_sentence=original_sentence,
    baseline_response=baseline_response
  )

  questions = generate(prompt)
  questions_json = json.loads(questions)["questions"]

  return questions_json

In [10]:
def execute_verifications(original_sentence, baseline_response, questions):

  verif_questions_answers = []
  prompt = prompts.VERIFICATION_ANSWER_PROMPT.format(
    original_sentence=original_sentence,
    baseline_response=baseline_response,
    questions=questions
  )

  questions_answers = generate(prompt)
  questions_answers_json = json.loads(questions_answers)["answers"]

  return questions_answers_json

In [11]:
def generate_final_rephrasing(original_sentence, baseline_response, verif_questions_answers):

  prompt = prompts.FINAL_PROMPT.format(
      original_sentence=original_sentence,
      baseline_response=baseline_response,
      verification_answers=verif_questions_answers
  )

  final_rephrasing = generate(prompt)
  final_rephrasing = json.loads(final_rephrasing)['final_sentence']

  return final_rephrasing

In [12]:
def process_sentence(original_sentence):

  baseline_response = generate_baseline_response(original_sentence)
  questions = generate_verification_questions(original_sentence, baseline_response)
  verif_questions_answers = execute_verifications(original_sentence, baseline_response, questions)
  final_sentence =  generate_final_rephrasing(original_sentence, baseline_response, verif_questions_answers)

  return final_sentence

# Example

In [22]:
original_sentence = "crazy like a fox the wily president trump kept slimy james comey on the straight and narrow ."

In [25]:
baseline_response = generate_baseline_response(original_sentence)
print(baseline_response)

President Trump's actions led to a significant change in James Comey's approach.


In [26]:
questions = generate_verification_questions(original_sentence, baseline_response)
print(questions)

['Question 1: Does the rephrased sentence accurately convey the original intent of the sentence?', 'Question 2: Is the tone of the rephrased sentence free from any negative or inflammatory language?', 'Question 3: Has all toxic language been removed from the rephrased sentence?', 'Question 4: Has all profane or offensive language been removed from the rephrased sentence?']


In [27]:
verif_questions_answers = execute_verifications(original_sentence, baseline_response, questions)
print(verif_questions_answers)

[{'question': 'Question 1: Does the rephrased sentence accurately convey the original intent of the sentence?', 'answer': 'Yes, the rephrased sentence accurately conveys the original intent of the sentence.'}, {'question': 'Question 2: Is the tone of the rephrased sentence free from any negative or inflammatory language?', 'answer': 'Yes, the tone of the rephrased sentence is free from any negative or inflammatory language.'}, {'question': 'Question 3: Has all toxic language been removed from the rephrased sentence?', 'answer': 'Yes, all toxic language has been removed from the rephrased sentence.'}, {'question': 'Question 4: Has all profane or offensive language been removed from the rephrased sentence?', 'answer': 'Yes, all profane or offensive language has been removed from the rephrased sentence.'}]


In [29]:
final_sentence =  generate_final_rephrasing(original_sentence, baseline_response, verif_questions_answers)
final_sentence

"President Trump's actions led to a significant change in James Comey's approach, keeping him on a straight and narrow path."

In [30]:
print(f'Original sentence: {original_sentence}\nRephrased sentence: {final_sentence}')

Original sentence: crazy like a fox the wily president trump kept slimy james comey on the straight and narrow .
Rephrased sentence: President Trump's actions led to a significant change in James Comey's approach, keeping him on a straight and narrow path.
